# Домашнее задание "Функции потерь и оптимизация"

In [1]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import train_test_split


## Подготовка датасета

In [2]:
data = load_iris()

In [3]:
data.target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [4]:
data.data.shape

(150, 4)

In [5]:
data.target.shape

(150,)

Оставим в датасете записи, связанные с классами **Iris Versicolor** и **Iris Virginica**

In [6]:
data.target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [7]:
X = data.data[(data.target == 1) | (data.target == 2)]
X.shape

(100, 4)

Перекодируем целевую переменную

In [8]:
y = data.target[(data.target == 1) | (data.target == 2)]
y[y == 1] = 0
y[y == 2] = 1
y.shape

(100,)

In [9]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

Добавим дополнительный еденичный признак

In [10]:
X = np.c_[np.ones(len(X)), X]
X[:5]

array([[1. , 7. , 3.2, 4.7, 1.4],
       [1. , 6.4, 3.2, 4.5, 1.5],
       [1. , 6.9, 3.1, 4.9, 1.5],
       [1. , 5.5, 2.3, 4. , 1.3],
       [1. , 6.5, 2.8, 4.6, 1.5]])

In [11]:
X.shape

(100, 5)

Разделим датасет на обучающую и тестовую выборки

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1, stratify=y)

## Собственная реализация логистической регрессии с реализацией градиентного спуска

In [13]:
class LogisticRegressionGD(object):
    
    def __init__(self, eta=0.05, n_iter=100, random_state=1):
        self.eta = eta
        self.n_iter = n_iter
        self.random_state = random_state

    
    def fit(self, X, y):
        self.w_ = np.random.randn(X.shape[1])
        # пустой список издержек:
        self.loss_ = []
        for i in range(self.n_iter):
            output = self.activation(X)
            errors = (y - output)
            self.w_ += self.eta * X.T.dot(errors)
            loss = -(y.dot(np.log(output + 1e-30)) +
                   ((1 - y).dot(np.log((1 - output) + 1e-30)))) / X.shape[0]
            self.loss_.append(loss)
        return self


    def net_input(self, X):
        return np.dot(X, self.w_)

    
    def activation(self, z):
        return 1.0 / (1.0 + np.exp(-self.net_input(z)))

    
    def predict(self, X):
        return np.where(self.activation(X) >= 0.5, 1, 0)


    def accuracy(self, X, y):
        return (self.predict(X) == y).sum() / len(y)

In [14]:
model = LogisticRegressionGD(eta=0.05, n_iter=1000, random_state=1)

In [15]:
model.fit(X_train, y_train)

In [16]:
model.activation(X_test)

array([1.00000000e+00, 1.00000000e+00, 9.14952783e-26, 8.43444878e-14,
       1.66636886e-16, 9.99999998e-01, 4.39093920e-02, 1.00000000e+00,
       9.89659690e-25, 1.05152864e-27, 1.00000000e+00, 1.00000000e+00,
       1.00000000e+00, 3.00105322e-16, 1.00000000e+00, 1.00000000e+00,
       1.24038181e-11, 1.00000000e+00, 6.07685085e-23, 1.96821851e-08,
       1.83586025e-30, 5.37119117e-27, 1.91054456e-19, 9.99979979e-01,
       1.00000000e+00, 1.05539364e-29, 3.27264046e-24, 1.00000000e+00,
       9.99999859e-01, 3.08964207e-27])

In [17]:
model.predict(X_test)

array([1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 1, 1, 0])

In [18]:
y_test

array([1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 1, 1, 0])

In [19]:
model.accuracy(X_test, y_test)

0.9666666666666667